In [ ]:
#|default_exp data_utils 

# Datasets and General Data Utilities

For many tasks in machine learning, the actual model training is the easy bit! Many data scientists spend most of their time sourcing and exploring data, and getting it into the right format ready for modelling. Lucky for us, most of the data we'll use for demos during this course has already been collected and organised for us, and to make things even more convenient during the lessons themselves we're going to lay some additional groundwork here in this notebook. 

Motivate dataloaders
Batching
Advantage of pre-fetching the next batch
Mention monitoring GPU usage and watching for CPU bottlenecks in the dataloaders
Dive into pytorch dataloaders
HF Hub and datasets library

https://huggingface.co/docs/datasets/quickstart


https://huggingface.co/docs/datasets/stream

DATA UTILS 

In [ ]:
#|export
import torch
import datasets
from tglcourse.utils import *
from datasets import load_dataset
from torchvision import transforms as T
from torch.utils.data import Dataset, DataLoader

### MNIST

Describe

In [ ]:
#|eval: false

mnist_dataset = load_dataset('mnist') # It's on HF
mnist_dataset

Reusing dataset mnist (/home/jonathan/.cache/huggingface/datasets/mnist/mnist/1.0.0/fda16c03c4ecfb13f165ba7e29cf38129ce035011519968cdaf74894ce91c9d4)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})

In [ ]:
#|eval: false

# View the label and image for the first example:
print(mnist_dataset['train'][0]['label'])
mnist_dataset['train'][0]['image']

5


In [ ]:
#|eval: false

# Create a dataloader, with some pre-processing:

# A function to get our data in the right format: in this case just converting the image to a tensor
def transform(example):
    example["image"] = [T.ToTensor()(image) for image in example["image"]]
    return example

# Add this as a transform to the dataset
mnist_dataset = mnist_dataset.with_transform(transform)

# Create a pytorch dataloader with this, specifically just the 'train' set
dataloader = DataLoader(mnist_dataset['train'], batch_size=4)

# Get one batch of data:
images, labels = next(iter(dataloader)).values() # It returns a dict by default, .values gets just the data
images.shape, labels

(torch.Size([4, 1, 28, 28]), tensor([5, 0, 4, 1]))

Another way to do this would be to define our own dataset class which needs to implement `__init__`, `__len__` and `__getitem__`:

In [ ]:
#|eval: false

class MNIST_DATASET(Dataset):
    """One option: custom Dataset class"""
    def __init__(self, split='train'):
        self.dataset = load_dataset('mnist', split=split)
        self.preprocess = T.ToTensor()
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, idx):
        x = self.dataset[idx]
        return (self.preprocess(x['image']), x['label'])
        
dataset = MNIST_DATASET(split='train')
dataloader = DataLoader(dataset, batch_size=8)
images, labels = next(iter(dataloader))
images.shape, labels

Reusing dataset mnist (/home/jonathan/.cache/huggingface/datasets/mnist/mnist/1.0.0/fda16c03c4ecfb13f165ba7e29cf38129ce035011519968cdaf74894ce91c9d4)


(torch.Size([8, 1, 28, 28]), tensor([5, 0, 4, 1, 9, 2, 1, 3]))

## Streaming

In [ ]:
#|eval: false

# Load the dataset from HF with streaming=True this time
mnist_dataset = load_dataset('mnist', split='train', streaming=True) 

# Apply the pre-processing
mnist_dataset = mnist_dataset.map(transform, batched=True)

# Format for pytorch
mnist_dataset = mnist_dataset.with_format("torch")

# Shuffle the dataset (optional, buffer size is how much is loaded to shuffle
# mnist_dataset = mnist_dataset.shuffle(seed=42, buffer_size=500)

# Create the dataloader, with the transforms passed as the collate_fn
dataloader = DataLoader(mnist_dataset, batch_size=4)

# Now we can get a batch as usual, but it doesn't need to download anything in advance!
images, labels = next(iter(dataloader)).values()
print(images.shape, labels)

torch.Size([4, 1, 28, 28]) tensor([5, 0, 4, 1])


## Specifying some datasets for export

In [ ]:
#|export

# Specify the pre-processing
to_tensor = T.ToTensor()
def mnist_transform(example):
    example["image"] = [to_tensor(image) for image in example["image"]]
    return example

# Re-create the streaming example above
def get_mnist_dl(batch_size=32, streaming=True):
    mnist_dataset = load_dataset('mnist', split='train', streaming=streaming)
    if streaming:
        mnist_dataset = mnist_dataset.map(mnist_transform, batched=True)
        mnist_dataset = mnist_dataset.with_format("torch")
    else:
        mnist_dataset = mnist_dataset.with_transform(transform)
    dataloader = DataLoader(mnist_dataset, batch_size=batch_size)
    return dataloader

In [ ]:
# Test this
dl = get_mnist_dl()
images, labels = next(iter(dl)).values()
print(images.shape, labels)

torch.Size([32, 1, 28, 28]) tensor([5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1, 7, 2, 8, 6, 9, 4, 0, 9, 1,
        1, 2, 4, 3, 2, 7, 3, 8])


In [ ]:
#|eval: false
%%time
for images, labels in dl:
    a = images*2

CPU times: user 17.4 s, sys: 77.1 ms, total: 17.5 s
Wall time: 22.3 s


In [ ]:
#|eval: false
# We can pass streaming=False here to force the download of the whole dataset:
dl = get_mnist_dl(streaming=False)
images, labels = next(iter(dl)).values()
print(images.shape, labels)

Reusing dataset mnist (/home/jonathan/.cache/huggingface/datasets/mnist/mnist/1.0.0/fda16c03c4ecfb13f165ba7e29cf38129ce035011519968cdaf74894ce91c9d4)


torch.Size([32, 1, 28, 28]) tensor([5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1, 7, 2, 8, 6, 9, 4, 0, 9, 1,
        1, 2, 4, 3, 2, 7, 3, 8])


In [ ]:
#|eval: false
%%time
for images, labels in dl:
    a = images*2

CPU times: user 13.4 s, sys: 0 ns, total: 13.4 s
Wall time: 13.4 s


In [ ]:
# TODO celebA and imagewoof from https://github.com/johnowhitaker/cclddg/blob/master/02_Datasets.ipynb